In [1]:
import sys
import os
import threading
import time
import json
from kafka3 import KafkaProducer, KafkaConsumer
from datetime import datetime as dt
from pymongo import MongoClient
from pyspark.sql import Row
import pandas as pd
import os
# add the folder where util.py lives
sys.path.append(os.path.join(os.getcwd(), 'fit3182-a2'))
from util import kafkaProducer

In [2]:
from kafka3.admin import KafkaAdminClient, NewTopic
import time

def recreate_topics(bootstrap_servers, topics, num_partitions=1, replication_factor=1):
    admin = KafkaAdminClient(bootstrap_servers=bootstrap_servers)
    # 1. Delete
    try:
        admin.delete_topics(topics)
        print(f"[INFO] Deletion of topics {topics} initiated.")
    except Exception as e:
        print(f"[WARN] Could not delete topics (they may not exist yet): {e}")
    # Kafka needs a moment to finish deletion
    time.sleep(2)

    # 2. Recreate
    new_topics = [NewTopic(name=topic,
                           num_partitions=num_partitions,
                           replication_factor=replication_factor)
                  for topic in topics]
    try:
        admin.create_topics(new_topics)
        print(f"[INFO] Recreated topics {topics}.")
    except Exception as e:
        print(f"[ERROR] Failed to create topics {topics}: {e}")
        raise
    finally:
        admin.close()

In [ ]:
csv_path_a="data/camera_event_A.csv"
csv_path_b="data/camera_event_B.csv"
csv_path_c="data/camera_event_C.csv"

kafka_server="172.22.32.1:9092"

producer_id_a="A"
producer_id_b="B"
producer_id_c="C"

topic_a="camera_event_a"
topic_b="camera_event_b"
topic_c="camera_event_c"

batch_interval=5

topics = ["camera_event_a", "camera_event_b", "camera_event_c"]

# PURGE & RECREATE before producing
recreate_topics(kafka_server, topics,
                num_partitions=1,
                replication_factor=1)

    
producer_a = kafkaProducer(csv_path_a, kafka_server, producer_id_a, topic_a, batch_interval)
producer_b = kafkaProducer(csv_path_b, kafka_server, producer_id_b, topic_b, batch_interval)
producer_c = kafkaProducer(csv_path_c, kafka_server, producer_id_c, topic_c, batch_interval)

threads = []
for p in (producer_a, producer_b, producer_c):
    t = threading.Thread(target=p.produce_batches, daemon=True)
    t.start()
    threads.append(t)

for t in threads:
    t.join()

[INFO] Deletion of topics ['camera_event_a', 'camera_event_b', 'camera_event_c'] initiated.
[INFO] Recreated topics ['camera_event_a', 'camera_event_b', 'camera_event_c'].
[INFO] Successfully loaded data from data/camera_event_A.csv
print 172.22.32.1:9092
[INFO] Successfully loaded data from data/camera_event_B.csv
print 172.22.32.1:9092
[INFO] Successfully loaded data from data/camera_event_C.csv
print 172.22.32.1:9092
172.22.32.1:9092
172.22.32.1:9092
172.22.32.1:9092
[INFO] Publishing batch #1 (20 records)...
[INFO] Publishing batch #1 (1 records)...
[INFO] Publishing batch #1 (5 records)...
[INFO] Batch #1 sent. Sleeping 5s...
[INFO] Batch #1 sent. Sleeping 5s...
[DATA] Batch #1:
                               event_id  batch_id car_plate  camera_id  \
0  d86e8cdb-c387-4ccb-b35a-346302238824         1   UTT 229          3   

                   timestamp  speed_reading  
0 2024-01-01 08:00:54.958092          130.8  

[DATA] Batch #1:
                                event_id  batch_

[INFO] Publishing batch #4 (2 records)...
[INFO] Publishing batch #4 (20 records)...
[INFO] Publishing batch #4 (4 records)...
[INFO] Batch #4 sent. Sleeping 5s...
[DATA] Batch #4:
                               event_id  batch_id car_plate  camera_id  \
4  55a60e1c-818e-4e55-bc02-0627765629b2         4     ICE 8          3   
5  daafd000-91fa-4ff7-a9bf-896ee74b35ad         4     WC 89          3   

                   timestamp  speed_reading  
4 2024-01-01 08:01:16.405088          108.0  
5 2024-01-01 08:01:18.835653           94.5  

[INFO] Batch #4 sent. Sleeping 5s...
[DATA] Batch #4:
                                event_id  batch_id car_plate  camera_id  \
60  1dd988c0-d000-489b-8cf0-2cd738fb0565         4    BQN 88          1   
61  9478323b-a2f6-4592-8c9f-a9de5bb984ab         4  ZEA 3530          1   
62  3e613964-b717-44a8-bf81-a1290ae07a35         4     SJ 15          1   
63  9f538681-69f0-4efd-a3e3-ef44ee2c1094         4     PB 55          1   
64  26e1dd39-29ec-4692-bca0-

[INFO] Publishing batch #7 (2 records)...
[INFO] Batch #7 sent. Sleeping 5s...
[DATA] Batch #7:
                               event_id  batch_id car_plate  camera_id  \
8  cc084bd6-f680-43cf-9d6e-15bc481038b2         7  PKH 8115          3   
9  676fa9f2-4a5b-4d1b-a63e-a59b1f2e5c26         7      PI 9          3   

                   timestamp  speed_reading  
8 2024-01-01 08:08:52.583094          153.8  
9 2024-01-01 08:08:53.620450          134.3  

[INFO] Publishing batch #7 (3 records)...
[INFO] Publishing batch #7 (20 records)...
[INFO] Batch #7 sent. Sleeping 5s...
[DATA] Batch #7:
                                event_id  batch_id car_plate  camera_id  \
17  7dff112a-d3ce-4ebb-841f-ea3e03662ead         7    WB 418          2   
18  1f739fcb-8e46-4c92-a69b-9a18533cb646         7      PI 9          2   
19  756a47f3-7f81-4e17-88fa-b2bbed9aa29e         7  PKH 8115          2   

                    timestamp  speed_reading  
17 2024-01-01 08:08:24.091422          138.8  
18 2024-

[INFO] Publishing batch #10 (2 records)...
[INFO] Batch #10 sent. Sleeping 5s...
[DATA] Batch #10:
                                event_id  batch_id car_plate  camera_id  \
13  2f38e970-5042-4d91-8ef3-f8e4bed9908f        10    CZ 592          3   
14  fa9e3b2a-de4e-48be-b97b-1c01e95e437a        10      AH 8          3   

                    timestamp  speed_reading  
13 2024-01-01 08:00:47.453301          161.4  
14 2024-01-01 08:00:49.247531          165.5  

[INFO] Publishing batch #10 (2 records)...
[INFO] Batch #10 sent. Sleeping 5s...
[DATA] Batch #10:
                                event_id  batch_id car_plate  camera_id  \
23  58588fd9-1fd6-4b52-929b-406731bbce2b        10  WPV 1456          2   
24  274dffed-ed69-4f56-8a2e-5c017d27adc1        10     QJ 53          2   

                    timestamp  speed_reading  
23 2024-01-01 08:08:45.845657           75.1  
24 2024-01-01 08:08:46.302895           82.8  

[INFO] Publishing batch #10 (20 records)...
[INFO] Batch #10 sent.

[INFO] Publishing batch #13 (1 records)...
[INFO] Batch #13 sent. Sleeping 5s...
[DATA] Batch #13:
                                event_id  batch_id car_plate  camera_id  \
17  49500aa0-39b9-4dc2-8e4d-69c8519b5983        13     JEJ 6          3   

                    timestamp  speed_reading  
17 2024-01-01 08:09:06.369311          109.3  

[INFO] Publishing batch #13 (1 records)...
[INFO] Batch #13 sent. Sleeping 5s...
[DATA] Batch #13:
                                event_id  batch_id car_plate  camera_id  \
27  7dba46aa-77b5-47f5-9801-d30e5ddff0d7        13     FO 32          2   

                    timestamp  speed_reading  
27 2024-01-01 08:00:39.240776           94.8  

[INFO] Publishing batch #13 (20 records)...
[INFO] Batch #13 sent. Sleeping 5s...
[DATA] Batch #13:
                                 event_id  batch_id car_plate  camera_id  \
240  2e8b5c14-1d48-484c-8b3e-9500ee86717e        13    HM 258          1   
241  2b480838-5811-4479-b618-23f9d639eb22        13   WG 9

...

[INFO] Publishing batch #293 (2 records)...
[INFO] Batch #293 sent. Sleeping 5s...
[DATA] Batch #293:
                                 event_id  batch_id car_plate  camera_id  \
391  4e4d1eb4-896d-483e-a640-8ca31a675492       293    PB 116          3   
392  ac24be2e-e316-4d21-a9f3-1a938efc0c69       293    VHM 34          3   

                     timestamp  speed_reading  
391 2024-01-01 10:37:23.418510          110.6  
392 2024-01-01 10:37:25.391770          110.8  

[INFO] Publishing batch #293 (2 records)...
[INFO] Batch #293 sent. Sleeping 5s...
[DATA] Batch #293:
                                 event_id  batch_id car_plate  camera_id  \
568  29c893fd-473c-4f30-b408-1aa1c1b0268e       293   RM 5566          2   
569  051c634b-9c1f-4a76-831b-f2c8a7a30174       293    AC 372          2   

                     timestamp  speed_reading  
568 2024-01-01 11:34:47.742717          100.6  
569 2024-01-01 11:34:52.178963           86.5  

[INFO] Publishing batch #293 (20 records)...
[IN

[INFO] Publishing batch #296 (1 records)...
[INFO] Batch #296 sent. Sleeping 5s...
[DATA] Batch #296:
                                 event_id  batch_id car_plate  camera_id  \
395  d6310005-621b-4f40-b7a2-8f40e717dcae       296    ZMB 31          3   

                     timestamp  speed_reading  
395 2024-01-01 10:37:46.710860           81.5  

[INFO] Publishing batch #296 (3 records)...
[INFO] Batch #296 sent. Sleeping 5s...
[DATA] Batch #296:
                                 event_id  batch_id car_plate  camera_id  \
572  f4264729-01f8-4116-aa23-3788f78f8f69       296    OEQ 64          2   
573  2c574884-758d-4727-b4a8-e649a38f5f23       296      CN 9          2   
574  fee57dda-e3d3-427d-ae79-c261810426dc       296   EZX 371          2   

                     timestamp  speed_reading  
572 2024-01-01 11:40:27.141011          147.9  
573 2024-01-01 11:40:29.943376          119.4  
574 2024-01-01 11:40:30.264377          134.1  

[INFO] Publishing batch #296 (20 records)...
[IN

[INFO] Publishing batch #299 (1 records)...
[INFO] Batch #299 sent. Sleeping 5s...
[DATA] Batch #299:
                                 event_id  batch_id car_plate  camera_id  \
398  778b1499-9e0f-4323-9ff6-3b89151b349d       299  DLH 2835          3   

                     timestamp  speed_reading  
398 2024-01-01 10:44:26.810060          137.4  

[INFO] Publishing batch #299 (2 records)...
[INFO] Batch #299 sent. Sleeping 5s...
[DATA] Batch #299:
                                 event_id  batch_id car_plate  camera_id  \
577  a5d68f1f-7a4d-4e3f-a51c-5a927cfae4e5       299   UUK 911          2   
578  4c85a437-537c-4daa-a277-cfc673f3ff19       299   WKL 349          2   

                     timestamp  speed_reading  
577 2024-01-01 11:40:55.279705           62.5  
578 2024-01-01 11:40:58.330998           62.0  

[INFO] Publishing batch #299 (20 records)...
[INFO] Batch #299 sent. Sleeping 5s...
[DATA] Batch #299:
                                  event_id  batch_id car_plate  camer

[INFO] Publishing batch #302 (1 records)...
[INFO] Batch #302 sent. Sleeping 5s...
[DATA] Batch #302:
                                 event_id  batch_id car_plate  camera_id  \
402  583d8d14-2215-435a-a04f-4b07dae426e3       302  OUV 7317          3   

                     timestamp  speed_reading  
402 2024-01-01 10:20:38.850869           84.1  

[INFO] Publishing batch #302 (2 records)...
[INFO] Batch #302 sent. Sleeping 5s...
[DATA] Batch #302:
                                 event_id  batch_id car_plate  camera_id  \
584  eea9862d-f498-42c9-b715-5ab9588e200a       302    HHC 65          2   
585  450b7acc-aca2-4762-8bab-93aaafabf27f       302    XSH 84          2   

                     timestamp  speed_reading  
584 2024-01-01 11:34:42.648486          103.1  
585 2024-01-01 11:34:45.289305          110.8  

[INFO] Publishing batch #302 (20 records)...
[INFO] Batch #302 sent. Sleeping 5s...
[DATA] Batch #302:
                                  event_id  batch_id car_plate  camer

[INFO] Publishing batch #305 (20 records)...
[INFO] Batch #305 sent. Sleeping 5s...
[DATA] Batch #305:
                                  event_id  batch_id car_plate  camera_id  \
6080  0eb0df9a-772b-47d4-bf43-ae958ff88591       305  MGH 0724          1   
6081  f970ce5f-de8e-4560-a875-ff9126a37c6d       305     UB 49          1   
6082  89c86df1-d580-4182-bb0a-c3d62e331847       305    UA 229          1   
6083  721628b2-d0a5-4f59-ac43-923fe065153b       305      TC 7          1   
6084  aad263c8-9184-4d45-84a6-d7978215234c       305     SAL 8          1   
6085  d812fe18-5afa-45e8-985b-c6d84e638130       305   FT 9963          1   
6086  7a0b9bcb-5924-4aeb-b499-cc781573d393       305    TJZ 83          1   
6087  4a642acb-3bd8-47a6-8daf-d970f971f3b6       305      VL 5          1   
6088  1f255e6b-f93c-4706-a47b-dad3c36e0818       305    JD 855          1   
6089  b3c15b8e-25b6-4571-8ab3-f431aa19eced       305     OJS 7          1   
6090  b8755361-e5ad-49da-9afc-c8040e55f45e       3